In [3]:
import rosbag
import tf
from geometry_msgs import msg
import geometry_msgs.msg

import tf_conversions.posemath as posemath
import numpy as np

In [4]:
object_child_frame_id = "object_link"
base_child_frame_id = "base_link"

http://www.euclideanspace.com/maths/geometry/rotations/conversions/eulerToMatrix/examples/index.htm

In [5]:
def applyTF2object(base, obj):

    q1 = np.zeros(4)
    q1[0] = base.transform.rotation.x
    q1[1] = base.transform.rotation.y
    q1[2] = base.transform.rotation.z
    q1[3] = base.transform.rotation.w
    P1 = tf.transformations.quaternion_matrix(q1)
    P1[0, 3] = base.transform.translation.x
    P1[1, 3] = base.transform.translation.y
    P1[2, 3] = base.transform.translation.z
    
    q2 = np.zeros(4)
    q2[0] = obj.transform.rotation.x
    q2[1] = obj.transform.rotation.y
    q2[2] = obj.transform.rotation.z
    q2[3] = obj.transform.rotation.w
    P2 = tf.transformations.quaternion_matrix(q2)
    P2[0, 3] = obj.transform.translation.x
    P2[1, 3] = obj.transform.translation.y
    P2[2, 3] = obj.transform.translation.z
    
    P_ = P1*P2

    q = tf.transformations.quaternion_from_matrix(P_)
    
    geo_msg = geometry_msgs.msg()
    
    geo_msg.transform.rotation.x = q[0]
    geo_msg.transform.rotation.y = q[1]
    geo_msg.transform.rotation.z = q[2]
    geo_msg.transform.rotation.w = q[3]
    geo_msg.transform.translation.x = P_[0, 3]
    geo_msg.transform.translation.y = P_[1, 3]
    geo_msg.transform.translation.z = P_[2, 3]
    
    return geo_msg
    
base_msg = None
object_msg = None

def applyTF2bag(input_bag, output_bag):
    isFirstTF = True
    with rosbag.Bag(output_bag, 'w') as outbag:
        for topic, msg, t in rosbag.Bag(input_bag).read_messages():
            if topic == "/tf":
                tf_msg = tf.msg.tfMessage()
                
                for geo_msg in msg.transforms:
                    if geo_msg.child_frame_id == object_child_frame_id:
                        base_msg = geo_msg
                    if geo_msg.child_frame_id == base_child_frame_id:
                        object_msg = geo_msg
                
                if ((base_msg is not None) or (object_msg is not None)):
                    geo_msg = applyTF2object()
                    tf_msg.transforms.append(geo_msg)
                    outbag.write('/tf', tf_msg, msg.header.stamp if msg._has_header else t)
            else:
                outbag.write(topic, msg, msg.header.stamp if msg._has_header else t)

In [7]:
bagfiles = ['recorded_data.bag']
path = '/home/ruffsl/Desktop/'
for bagfile in bagfiles:
    applyTF2bag(path+bagfile, path+'optical/optical.'+bagfile)